# LAC <=> RNC MAPPING ALL MSSs (NOKIA & ERICSSON)

## Import Modules

In [13]:
# Import modules (NOK_ERI_Toolkit) & others that will be needed along the program
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import NOK_ERI_Toolkit as NOK_ERI_Toolkit

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Main Program

### Initialize variables

In [14]:
# Initialize NOKIA, ERICSSON & COMMON variables

# NOKIA variables 
NOK_MSS_Name = '0'
NOK_MSS_Date = '0'
NOK_MSS_Time = '0'
RNC_NAME = '0'
RNC_MCC = '0'
RNC_MNC = '0'
RNC_ID = '0'
RNC_MUL_PLMN = '0'
RNC_STAT = '0'
RNC_OSTAT = '0'
RNC_LAC_MCC = '0'
RNC_LAC_MNC = '0'
RNC_LAC_NO = '0'

NOK_MSS_Pars = [NOK_MSS_Name, NOK_MSS_Date, NOK_MSS_Time]
RNC_Pars = [RNC_NAME, RNC_MCC, RNC_MNC, RNC_ID, RNC_MUL_PLMN, RNC_STAT, RNC_OSTAT]
RNC_LAC_Pars = [RNC_LAC_MCC, RNC_LAC_MNC, RNC_LAC_NO]

NOK_index = 0
NOK_position = 0
NOK_RNC_LAC_count = 0
NOK_RNC_LAC_flag = 0

# ERICSSON variables 
ERI_MSS_Name = '0'
ERI_MSS_Date = '0'
ERI_MSS_Time = '0'
RNC_NAME = '0'
RNC_LAC_MCC = '0'
RNC_LAC_MNC = '0'
RNC_LAC_NO = '0'

ERI_MSS_Pars = [ERI_MSS_Name, ERI_MSS_Date, ERI_MSS_Time]
RNC_Pars = [RNC_NAME]
RNC_LAC_Pars = [RNC_LAC_MCC, RNC_LAC_MNC, RNC_LAC_NO]

ERI_index = 0
ERI_position = 0
ERI_RNC_LAC_count = 0
ERI_RNC_LAC_flag = 0

In [15]:
# Create & Initialize & Set the NOKIA & ERICSSON pd.DataFrame

# NOKIA
NOK_col_names = ['MSS','Date','Time','RNC','RNC_MCC','RNC_MNC','RNC_ID','RNC_MULT_PLMN',
                'RNC_STAT','RNC_OP_STAT','MCC','MNC','LAC']
df_NOK_MSS_RNC_LACs = pd.DataFrame(columns=NOK_col_names)
df_NOK_MSS_RNC_LACs.loc[0] = [NOK_MSS_Name, NOK_MSS_Date, NOK_MSS_Time, RNC_NAME,
                            RNC_MCC, RNC_MNC, RNC_ID, RNC_MUL_PLMN, RNC_STAT, RNC_OSTAT,
                            RNC_LAC_MCC, RNC_LAC_MNC, RNC_LAC_NO]

# ERICSSON
ERI_col_names = ['MSS','Date','Time','RNC','MCC','MNC','LAC']
df_ERI_MSS_RNC_LACs = pd.DataFrame(columns=ERI_col_names)
df_ERI_MSS_RNC_LACs.loc[0] = [ERI_MSS_Name, ERI_MSS_Date, ERI_MSS_Time,
                            RNC_NAME, RNC_LAC_MCC, RNC_LAC_MNC, RNC_LAC_NO]

In [16]:
# Especify the path of the NOKIA & ERICSSON log files 
NOK_path_to_file = 'C:/Users/Alvaro/OneDrive/My Files/_Scripts_Models/Commands Generators Logs/20220204 - ZE2I - All_MSS.txt'
ERI_path_to_file = 'C:/Users/Alvaro/OneDrive/My Files/_Scripts_Models/Commands Generators Logs/20220204 - MSSBAxERICC - MGMAP.txt'

### Load log file

In [17]:
# Load the log file
NOK_log_file = NOK_ERI_Toolkit.open_file(NOK_path_to_file)
ERI_log_file = NOK_ERI_Toolkit.open_file(ERI_path_to_file)

### Execute NOKIA PROGRAM

In [18]:
# # Run the main program (txt => pd.DataFrame)
while (NOK_index < len(NOK_log_file)-1):
    NOK_index, NOK_position, NOK_MSS_Pars = NOK_ERI_Toolkit.get_mss_pars(NOK_log_file, NOK_index,'ZE2I::RT=ALL;')
    while (NOK_index < len(NOK_log_file)-1) and (not NOK_position == 999):
        NOK_index, NOK_position, RNC_Pars = NOK_ERI_Toolkit.get_ze2i_rnc_pars(NOK_log_file, NOK_index)
        while (NOK_index < len(NOK_log_file)-1) and (not NOK_position == 999):
            NOK_index, NOK_position, NOK_RNC_LAC_flag, RNC_LAC_Pars = NOK_ERI_Toolkit.get_ze2i_rnc_lac_pars(NOK_log_file, NOK_index, NOK_RNC_LAC_flag)
            if NOK_position == 999:                                                 # if 'COMMAND EXECUTED'
                NOK_index = NOK_index + 1                                           # Increase index to read next line
                break
            else:
                if (NOK_RNC_LAC_flag == 1):
                    new_RNC_LAC = NOK_MSS_Pars + RNC_Pars + RNC_LAC_Pars
                    df_NOK_MSS_RNC_LACs.loc[NOK_RNC_LAC_count] = new_RNC_LAC
                    NOK_RNC_LAC_count = NOK_RNC_LAC_count + 1
                else:
                    break

### Execute ERICSSON PROGRAM

In [19]:
# Run the main program (txt => pd.DataFrame)
while (ERI_index < len(ERI_log_file)-1):
    ERI_index, ERI_position, ERI_MSS_Pars = NOK_ERI_Toolkit.get_mss_pars(ERI_log_file, ERI_index,'eaw')
    while(ERI_position >= 0) and (not ERI_position == 998):
        ERI_index, ERI_position, RNC_Pars = NOK_ERI_Toolkit.get_mgmap_rnc_pars(ERI_log_file, ERI_index)
        if (ERI_position >= 0) and (not ERI_position == 999) and (not ERI_position == 998):
            ERI_index, ERI_position, RNC_LAC_Pars = NOK_ERI_Toolkit.get_mgmap_rnc_lac_pars(ERI_log_file, ERI_index)
            while(ERI_position >= 0) and (not ERI_position == 999) and (not ERI_position == 998):
                ERI_index, ERI_position, RNC_LAC_Pars = NOK_ERI_Toolkit.get_mgmap_rnc_lac_pars(ERI_log_file, ERI_index)
                if(ERI_position >= 0) and (not ERI_position == 999) and (not ERI_position == 998):
                    new_RNC_LAC = ERI_MSS_Pars + RNC_Pars + RNC_LAC_Pars
                    df_ERI_MSS_RNC_LACs.loc[ERI_RNC_LAC_count] = new_RNC_LAC
                    ERI_RNC_LAC_count = ERI_RNC_LAC_count + 1
                ERI_index = ERI_index + 1
        ERI_index = ERI_index + 1
    ERI_index = ERI_index + 1

### Drop columns & prepare to merge

In [20]:
# NOKIA DataFrame => Drop some columns to get just the most important ones.

df_NOK_MSS_RNC_LAIs = df_NOK_MSS_RNC_LACs
df_NOK_MSS_RNC_LAIs['MSS'] = df_NOK_MSS_RNC_LAIs['MSS'].str.replace(r'0', '')
df_NOK_MSS_RNC_LAIs['LAI'] =  df_NOK_MSS_RNC_LAIs['MCC'] + '-' + df_NOK_MSS_RNC_LAIs['MNC'] + '-' + df_NOK_MSS_RNC_LAIs['LAC']
df_NOK_MSS_RNC_LAIs['MSS-LAI'] =  df_NOK_MSS_RNC_LAIs['MSS'] + '-' + df_NOK_MSS_RNC_LAIs['LAI']
df_NOK_MSS_RNC_LAIs['NE-LAI'] =  df_NOK_MSS_RNC_LAIs['RNC'] + '-' + df_NOK_MSS_RNC_LAIs['LAI']

NOK_drop_col = ['RNC_MCC', 'RNC_MNC', 'RNC_MULT_PLMN', 'RNC_STAT', 'RNC_OP_STAT', 'RNC_ID']
df_NOK_MSS_RNC_LAIs = df_NOK_MSS_RNC_LAIs.drop(columns=NOK_drop_col)
NOK_col = ['MSS', 'Date', 'Time', 'MCC', 'MNC', 'LAC', 'LAI', 'MSS-LAI', 'NE-LAI', 'RNC']
df_NOK_MSS_RNC_LAIs = df_NOK_MSS_RNC_LAIs[NOK_col]

In [21]:
# ERICSSON DataFrame => Drop some columns to get just the most important ones.

df_ERI_MSS_RNC_LAIs = df_ERI_MSS_RNC_LACs
df_ERI_MSS_RNC_LAIs['LAI'] =  df_ERI_MSS_RNC_LAIs['MCC'] + '-' + df_ERI_MSS_RNC_LAIs['MNC'] + '-' + df_ERI_MSS_RNC_LAIs['LAC']
df_ERI_MSS_RNC_LAIs['MSS-LAI'] =  df_ERI_MSS_RNC_LAIs['MSS'] + '-' + df_ERI_MSS_RNC_LAIs['LAI']
df_ERI_MSS_RNC_LAIs['NE-LAI'] =  df_ERI_MSS_RNC_LAIs['RNC'] + '-' + df_ERI_MSS_RNC_LAIs['LAI']

ERI_drop_col = []
df_ERI_MSS_RNC_LAIs = df_ERI_MSS_RNC_LAIs.drop(columns=ERI_drop_col)
ERI_col = ['MSS', 'Date', 'Time', 'MCC', 'MNC', 'LAC', 'LAI', 'MSS-LAI', 'NE-LAI', 'RNC']
df_ERI_MSS_RNC_LAIs = df_ERI_MSS_RNC_LAIs[ERI_col]

### Merge & Sort & Print

In [22]:
# Merge & Sort & Print NOKIA and ERICSSON.

df_NOK_ERI_MSS_LACs = df_NOK_MSS_RNC_LAIs.append(df_ERI_MSS_RNC_LAIs, ignore_index=True)
df_NOK_ERI_MSS_LACs = df_NOK_ERI_MSS_LACs.sort_values(by=['MSS','LAC'], ignore_index=True)
df_NOK_ERI_MSS_LACs

,MSS,Date,Time,MCC,MNC,LAC,LAI,MSS-LAI,NE-LAI,RNC
0,MSSBA3,0000-00-00,00:00:00,724,05,30171,724-05-30171,MSSBA3-724-05-30171,RNCBA01-724-05-30171,RNCBA01
1,MSSBA3,0000-00-00,00:00:00,724,05,30173,724-05-30173,MSSBA3-724-05-30173,RNCBA01-724-05-30173,RNCBA01
2,MSSBA3,0000-00-00,00:00:00,724,05,30174,724-05-30174,MSSBA3-724-05-30174,RNCBA01-724-05-30174,RNCBA01
3,MSSBA3,0000-00-00,00:00:00,724,05,30175,724-05-30175,MSSBA3-724-05-30175,RNCBA01-724-05-30175,RNCBA01
4,MSSBA3,0000-00-00,00:00:00,724,05,30177,724-05-30177,MSSBA3-724-05-30177,RNCBA01-724-05-30177,RNCBA01
...,...,...,...,...,...,...,...,...,...,...
340,MSSBAA,2022-02-04,09:46:14,724,11,60071,724-11-60071,MSSBAA-724-11-60071,RNCTCSA-724-11-60071,RNCTCSA
341,MSSBAA,2022-02-04,09:46:14,724,11,60071,724-11-60071,MSSBAA-724-11-60071,RNCTCSD-724-11-60071,RNCTCSD
342,MSSBAA,2022-02-04,09:46:14,724,11,60071,724-11-60071,MSSBAA-724-11-60071,RNCTCSB-724-11-60071,RNCTCSB
343,MSSBAA,2022-02-04,09:46:14,724,05,60274,724-05-60274,MSSBAA-724-05-60274,RNCBA04-724-05-60274,RNCBA04


### Plot + Insights

In [23]:
# Continue...